In [1]:
import sys 
sys.path.append("..") 
import pickle

import pandas as pd
import matplotlib.pyplot as plt

from utils import data_helper,result_helper
from models import lstm_model

In [2]:
train_ds2='../data/train/unlabeled_data_labeled_v1.csv'
train_df2=pd.read_csv(train_ds2)
train_df2

,cutted_content,class_label,content,by_words
0,侍魂 胧 月 传说 12 月 4 日 迎来 一波 维护 更新 值得一提的是 本次 更新 会 ...,游戏,侍魂胧月传说在今天12月4日迎来了一波维护更新，值得一提的是在本次更新以后就将会迎来周年庆哦...,通关 礼包
1,DNF 游戏 中 奥丁 启示 光环 玩家 都 很 喜欢 一个 光环 装扮 外观 上 看 好看...,游戏,DNF游戏中的奥丁的启示光环是玩家们都很喜欢的一个重要的光环装扮，从外观上看是非常好看的，奥...,通关 礼包
2,剑 远征 中 一个 角色 都 一个 卡牌 角色 卡牌 强度 都 额 小伙伴 不 卡牌 强度 ...,游戏,在剑与远征中，每一个角色其实都是一个卡牌的，不过不同的角色所属于的卡牌强度都是不同的额，不过...,通关 礼包
3,阴阳师 圣诞节 这天 更新 想 玩家 一个 惊喜 式 神 陪 玩家 圣诞节 很多 小伙伴 长...,游戏,阴阳师在圣诞节这天更新也是想给玩家一个惊喜吧，式神陪着玩家们过圣诞节，不过很多小伙伴对于长篇...,通关 礼包
4,云梦 四时 歌 游戏 中 星宿 搭配 这款 游戏 中 最 阵容 搭配 强度 强度 越大 通关...,游戏,云梦四时歌游戏中的星宿搭配是非常重要的，这款游戏中最主要的就是阵容搭配的强度，强度越大那么通...,通关 礼包
...,...,...,...,...
8333,各支 俱乐部 贡献 多场 精彩 比赛 本期 邀请 四位 俱乐部 教练 介绍 俱乐部 情况 听...,体育,各支俱乐部为大家贡献了多场精彩的比赛，本期我们邀请到其中四位俱乐部的教练，为大家介绍本俱乐部...,队员 羽毛球
8334,9 月 3 日 2020 中国羽毛球 俱乐部 超级 联赛 第六个 比赛 日 全胜 青岛 仁洲...,体育,9月3日，2020中国羽毛球俱乐部超级联赛进入到第六个比赛日，谁能让全胜的青岛仁洲丢分成为最...,队员 羽毛球
8335,2020 中国羽毛球 俱乐部 超级 联赛 4 日晚 结束 常规赛 第一 循环 争夺 各支 俱...,体育,2020中国羽毛球俱乐部超级联赛4日晚结束了常规赛第一循环的争夺，各支俱乐部为大家贡献了多场...,队员 羽毛球
8336,2020 中国 光大 杯 中国羽毛球 俱乐部 超级 联赛 第六个 比赛 日 青岛 仁洲 41...,体育,2020“中国光大杯”中国羽毛球俱乐部超级联赛进入第六个比赛日，青岛仁洲4：1力克厦门特房，...,队员 羽毛球


In [3]:
train_ds1='../data/train/labeled_data_processed.csv'
train_df1=pd.read_csv(train_ds1)
train_df1

,id,class_label,content,text_len,label,cleaned_content,cutted_content
0,0,教育,人民网北京8月31日电（孙竞）日前，为进一步规范高等学校命名工作，依据相关法律法规规定，结合...,692,2,人民网北京8月31日电孙竞日前为进一步规范高等学校命名工作依据相关法律法规规定结合高校设置工...,人民网 北京 8 月 31 日电 孙竞 日前 进一步 规范 高等学校 命名 工作 依据 相关...
1,1,家居,又是一年落叶黄，一阵秋雨一阵凉；整日奔波工作忙，出门别忘添衣裳。金秋时节，正值装修旺季，集美...,816,0,又是一年落叶黄一阵秋雨一阵凉整日奔波工作忙出门别忘添衣裳金秋时节正值装修旺季集美家居继续带消...,一年 落叶 黄 一阵 秋雨 一阵 凉 整日 奔波 工作 忙 出门 别忘添 衣裳 金秋 时节 ...
2,2,教育,作者：祝安顺（深圳大学饶宗颐文化研究院编审、副教授） 越来越多的名校毕业生入职教师行业，吸...,1379,2,作者祝安顺深圳大学饶宗颐文化研究院编审副教授越来越多的名校毕业生入职教师行业吸引他们的是什么...,作者 祝 安顺 深圳大学 饶宗 颐 文化 研究院 编审 副教授 越来越 名校 毕业生 入职 ...
3,3,时尚,虽然此前其有过与品牌如Madawell的合作系列，但这回她确认将推出自己名下的服装系列。在最...,232,3,虽然此前其有过与品牌如Madawell的合作系列但这回她确认将推出自己名下的服装系列在最近的...,此前 其有 品牌 Madawell 合作 系列 这回 确认 推出 名下 服装 系列 采访 中...
4,4,教育,眼下，虽然疫情依然严峻，但我国大中小学都已陆续开学，疫情防控进入常态化，学生校园生活回归正常...,1740,2,眼下虽然疫情依然严峻但我国大中小学都已陆续开学疫情防控进入常态化学生校园生活回归正常尽管国内...,眼下 疫情 依然 严峻 我国 大中小学 都 已 陆续 开学 疫情 防控 常态 化 学生 校园...
...,...,...,...,...,...,...,...
6995,6995,教育,近期，国家卫健委发布《学校传染病症状监测预警技术指南》，其中规定，学校传染病达预警指标后，应...,1354,2,近期国家卫健委发布学校传染病症状监测预警技术指南其中规定学校传染病达预警指标后应立即对出现症...,近期 国家 卫健委 发布 学校 传染病 症状 监测 预警 技术 指南 学校 传染病 达 预警...
6996,6996,房产,新冠肺炎疫情影响之下，房企纷纷开设“线上售楼处”。据克而瑞监测，目前超过150家房企在做线上...,2819,1,新冠肺炎疫情影响之下房企纷纷开设线上售楼处据克而瑞监测目前超过150家房企在做线上营销其中超...,新冠 肺炎 疫情 影响 之下 房企 纷纷 开设 线上 售楼处 据克而瑞 监测 超过 150 ...
6997,6997,教育,学生犯错后，教师能不能惩戒？如何惩戒？惩戒的边界在哪里？9月起，《广东省学校安全条例》（以下...,1551,2,学生犯错后教师能不能惩戒如何惩戒惩戒的边界在哪里9月起广东省学校安全条例以下简称条例施行其中...,学生 犯错 后 教师 惩戒 惩戒 惩戒 边界 9 月 广东省 学校 条例 简称 条例 施行 ...
6998,6998,科技,6月3日下午，荣耀为我们带来了全新的荣耀Play4系列新品，包括了Play4和Play4Pr...,520,5,6月3日下午荣耀为我们带来了全新的荣耀Play4系列新品包括了Play4和Play4Pro两...,6 月 3 日 下午 荣耀 带来 全新 荣耀 Play4 系列 新品 包括 Play4 Pl...


In [4]:
class_label_to_class_dict = {
            0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经', 7: '游戏', 8: '娱乐', 9: '体育'}

class_to_class_label_dict={val:key for key,val in class_label_to_class_dict.items()}
class_to_class_label_dict

{'家居': 0,
 '房产': 1,
 '教育': 2,
 '时尚': 3,
 '时政': 4,
 '科技': 5,
 '财经': 6,
 '游戏': 7,
 '娱乐': 8,
 '体育': 9}

In [5]:
train_df2['label']=train_df2['class_label'].apply(lambda x:class_to_class_label_dict[x])
train_df2

,cutted_content,class_label,content,by_words,label
0,侍魂 胧 月 传说 12 月 4 日 迎来 一波 维护 更新 值得一提的是 本次 更新 会 ...,游戏,侍魂胧月传说在今天12月4日迎来了一波维护更新，值得一提的是在本次更新以后就将会迎来周年庆哦...,通关 礼包,7
1,DNF 游戏 中 奥丁 启示 光环 玩家 都 很 喜欢 一个 光环 装扮 外观 上 看 好看...,游戏,DNF游戏中的奥丁的启示光环是玩家们都很喜欢的一个重要的光环装扮，从外观上看是非常好看的，奥...,通关 礼包,7
2,剑 远征 中 一个 角色 都 一个 卡牌 角色 卡牌 强度 都 额 小伙伴 不 卡牌 强度 ...,游戏,在剑与远征中，每一个角色其实都是一个卡牌的，不过不同的角色所属于的卡牌强度都是不同的额，不过...,通关 礼包,7
3,阴阳师 圣诞节 这天 更新 想 玩家 一个 惊喜 式 神 陪 玩家 圣诞节 很多 小伙伴 长...,游戏,阴阳师在圣诞节这天更新也是想给玩家一个惊喜吧，式神陪着玩家们过圣诞节，不过很多小伙伴对于长篇...,通关 礼包,7
4,云梦 四时 歌 游戏 中 星宿 搭配 这款 游戏 中 最 阵容 搭配 强度 强度 越大 通关...,游戏,云梦四时歌游戏中的星宿搭配是非常重要的，这款游戏中最主要的就是阵容搭配的强度，强度越大那么通...,通关 礼包,7
...,...,...,...,...,...
8333,各支 俱乐部 贡献 多场 精彩 比赛 本期 邀请 四位 俱乐部 教练 介绍 俱乐部 情况 听...,体育,各支俱乐部为大家贡献了多场精彩的比赛，本期我们邀请到其中四位俱乐部的教练，为大家介绍本俱乐部...,队员 羽毛球,9
8334,9 月 3 日 2020 中国羽毛球 俱乐部 超级 联赛 第六个 比赛 日 全胜 青岛 仁洲...,体育,9月3日，2020中国羽毛球俱乐部超级联赛进入到第六个比赛日，谁能让全胜的青岛仁洲丢分成为最...,队员 羽毛球,9
8335,2020 中国羽毛球 俱乐部 超级 联赛 4 日晚 结束 常规赛 第一 循环 争夺 各支 俱...,体育,2020中国羽毛球俱乐部超级联赛4日晚结束了常规赛第一循环的争夺，各支俱乐部为大家贡献了多场...,队员 羽毛球,9
8336,2020 中国 光大 杯 中国羽毛球 俱乐部 超级 联赛 第六个 比赛 日 青岛 仁洲 41...,体育,2020“中国光大杯”中国羽毛球俱乐部超级联赛进入第六个比赛日，青岛仁洲4：1力克厦门特房，...,队员 羽毛球,9


In [6]:
# 打乱数据
train_df2=train_df2.sample(frac=1).reset_index(drop=True)
train_df2

,cutted_content,class_label,content,by_words,label
0,喜欢 一个 年代 创作 熟悉 陌生 张 亚东 评 皇后 皮箱 神秘 克制 儒雅 形容 张 亚...,娱乐,"“我们可以喜欢一个年代,但创作是既要熟悉,又要陌生的。”——张亚东评皇后皮箱 神秘,克制,儒...",电影 媒体,8
1,近日 电影 狐仙 大人 京 杀青 该片 北京 新 注力 文化 传播 有限公司 出品 北京 缘...,娱乐,近日，电影《hello狐仙大人》在京杀青，该片由北京新注力文化传播有限公司出品，北京缘润文化...,传媒 导演,8
2,好莱坞 冒险 灾难 大片 鲨海 逃生 1 月 10 日 全国 震撼 上映 47 米 深海 缺...,娱乐,好莱坞冒险灾难大片《鲨海逃生》将于1月10日全国震撼上映。47米深海缺氧环境、迷宫般的玛雅遗...,银幕 片中,8
3,每次 奥运会 最火 数 羽毛球 比赛 看着 运动员 挥舞 拍子 大汗淋漓 比赛 时 很多 人...,体育,每次的奥运会最火的数羽毛球比赛了，看着运动员挥舞拍子大汗淋漓的比赛时，很多人都会激起学羽毛球...,运动员 奥运会,9
4,大丰 号 北京 文化 有限公司 浙江 东阳 大有 文化 传播 有限公司 出品 北京 环纳 影...,娱乐,由大丰号(北京)文化有限公司、浙江东阳大有文化传播有限公司出品，北京环纳影业有限公司联合出品...,导演 影片,8
...,...,...,...,...,...
8333,斩获 六大 国际 电影节 27 项 大奖 电影 停战 日前 正式 发布 首款 忆 青春 版 ...,娱乐,斩获六大国际电影节27项大奖的电影《我们停战吧》日前正式发布首款“忆青春”版预告片，并宣布定...,影院 导演,8
8334,第五 人格 游戏 中 新 上线 一个 角色 日 活动 本次 活动 中有 一个 爱哭鬼 罗比 ...,游戏,第五人格游戏中最近新上线了一个角色日活动，本次活动中有一个爱哭鬼罗比生日纪念头像玩家们是可以...,玩家 攻略,7
8335,电影 老师 好 时光 主题 路演 火热 中 3 月 7 日 影片 导演 张栾 监制 兼 主演...,娱乐,电影《老师·好》“我们最好的时光”主题路演正在火热进行中。3月7日，影片导演张栾，监制兼主演...,传媒 主创,8
8336,崔斯韦 执导 张震 倪妮 廖凡 领衔主演 黄觉 刘桦 张奕聪 主演 李 光洁 特别 出演 首...,娱乐,由崔斯韦执导，张震、倪妮、廖凡领衔主演，黄觉、刘桦、张奕聪主演，李光洁特别出演的首部关注森林...,执导 主创,8


In [7]:
# 后三类每个保留1000条数据
train_df2_sub=pd.DataFrame()
for label in train_df2['label'].unique():
    train_df2_sub=train_df2_sub.append(train_df2[train_df2['label']==label].iloc[:1000])
train_df2_sub

,cutted_content,class_label,content,by_words,label
0,喜欢 一个 年代 创作 熟悉 陌生 张 亚东 评 皇后 皮箱 神秘 克制 儒雅 形容 张 亚...,娱乐,"“我们可以喜欢一个年代,但创作是既要熟悉,又要陌生的。”——张亚东评皇后皮箱 神秘,克制,儒...",电影 媒体,8
1,近日 电影 狐仙 大人 京 杀青 该片 北京 新 注力 文化 传播 有限公司 出品 北京 缘...,娱乐,近日，电影《hello狐仙大人》在京杀青，该片由北京新注力文化传播有限公司出品，北京缘润文化...,传媒 导演,8
2,好莱坞 冒险 灾难 大片 鲨海 逃生 1 月 10 日 全国 震撼 上映 47 米 深海 缺...,娱乐,好莱坞冒险灾难大片《鲨海逃生》将于1月10日全国震撼上映。47米深海缺氧环境、迷宫般的玛雅遗...,银幕 片中,8
4,大丰 号 北京 文化 有限公司 浙江 东阳 大有 文化 传播 有限公司 出品 北京 环纳 影...,娱乐,由大丰号(北京)文化有限公司、浙江东阳大有文化传播有限公司出品，北京环纳影业有限公司联合出品...,导演 影片,8
5,2017 年 9 月 17 日 腾讯 影业 年度 发布会 京 拉近 网娱 全资 附属 公司 ...,娱乐,2017年9月17日腾讯影业年度发布会在京召开，拉近网娱全资附属公司稼轩环球影业CEO钱重远...,主创 导演,8
...,...,...,...,...,...
2676,剑 远征 中 玩家 想要 提升 战斗力 经验 粉尘 角色 升级 装备 角色 提升 属性 种种...,游戏,在剑与远征中，玩家如果想要提升自己的战斗力的话，那么是需要获得经验粉尘给角色升级，还有装备给...,通关 礼包,7
2677,启源 女神 说 大火 很多 小伙伴 都 去 玩 游戏 才 玩 看着 琳琅满目 商品 难免 花...,游戏,最近启源女神可以说是大火啊，很多小伙伴都去玩这个游戏了，不过才开始玩的时候看着琳琅满目的商品...,礼包 副本,7
2678,万象 物语 一款 好玩 游戏 玩家 自由 组合 阵容 很多 新手 玩家 不 搭配 阵容 小编...,游戏,万象物语是一款非常好玩的游戏，玩家们可以在这里自由的组合阵容进行战斗，不过很多的新手玩家不知...,副本 玩家,7
2679,模拟 江湖 中 玩家 随从 帮组 提升 战斗力 日后 组建 门派 中是 很多 额 便利 小编...,游戏,模拟江湖中玩家们可以获得随从来帮组自己提升战斗力，并且在日后的组建门派中是有很多额便利的，小...,玩家 攻略,7


In [8]:
train_df=train_df1[['cutted_content','label']].append(train_df2_sub[['cutted_content','label','by_words']])
train_df

,cutted_content,label,by_words
0,人民网 北京 8 月 31 日电 孙竞 日前 进一步 规范 高等学校 命名 工作 依据 相关...,2,NaN
1,一年 落叶 黄 一阵 秋雨 一阵 凉 整日 奔波 工作 忙 出门 别忘添 衣裳 金秋 时节 ...,0,NaN
2,作者 祝 安顺 深圳大学 饶宗 颐 文化 研究院 编审 副教授 越来越 名校 毕业生 入职 ...,2,NaN
3,此前 其有 品牌 Madawell 合作 系列 这回 确认 推出 名下 服装 系列 采访 中...,3,NaN
4,眼下 疫情 依然 严峻 我国 大中小学 都 已 陆续 开学 疫情 防控 常态 化 学生 校园...,2,NaN
...,...,...,...
2676,剑 远征 中 玩家 想要 提升 战斗力 经验 粉尘 角色 升级 装备 角色 提升 属性 种种...,7,通关 礼包
2677,启源 女神 说 大火 很多 小伙伴 都 去 玩 游戏 才 玩 看着 琳琅满目 商品 难免 花...,7,礼包 副本
2678,万象 物语 一款 好玩 游戏 玩家 自由 组合 阵容 很多 新手 玩家 不 搭配 阵容 小编...,7,副本 玩家
2679,模拟 江湖 中 玩家 随从 帮组 提升 战斗力 日后 组建 门派 中是 很多 额 便利 小编...,7,玩家 攻略


In [9]:
train_df=train_df.sample(frac=1).reset_index(drop=True)
train_df['label'].value_counts()

7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: label, dtype: int64

In [10]:
train_df.to_csv('../data/train/train_processed.csv',index=False)